# IndoXTC - Fine-tune Head Prosa [XLM-R] [ALL]
Exploring Indonesian hate speech/abusive & sentiment text classification using multilingual language model.

This kernel is a part of my undergraduate final year project.
Checkout the full github repository:
https://github.com/ilhamfp/indonesian-text-classification-multilingual

In [1]:
#############################
# Experiment configurations #
#############################

INDO_DATA_NAME = 'prosa'
DATA_PATH_INDO = '../input/indoxtc-extracting-prosa-features-xlm-r'

FOREIGN_DATA_NAME = 'yelp'
DATA_PATH_FOREIGN = '../input/indoxtc-combining-yelp-features-xlm-r'

MODEL_NAME = 'XLM_R'

EXPERIMENT_TYPE_LIST = ['A', 'B', 'C'] # A / B / C
TOTAL_DATA_LIST = [500, 1000, 2500, 5000, 7500, 10981]
FOREIGN_LANG_DATA_MULT_LIST = [0.25, 0.5, 0.75, 1, 1.5, 2, 3, 4, 5, 6, 7, 8, 9, 10]
RANDOM_SEED_LIST = [4,5,6]
VALIDATION_DATA = 0.1

In [2]:
import os
import numpy as np
import pandas as pd
from load_data import load_experiment_features, set_random_seed_data
from model_head import set_seed, train, test, evaluate

CUDA is not available.  Training on CPU ...


## Experiment

In [3]:
result_list_final = []
for RANDOM_SEED in RANDOM_SEED_LIST:
    print("\n############################")
    print("### START RANDOM_SEED {} ###".format(RANDOM_SEED))
    print("############################")
    result_list = []
    for TOTAL_DATA in TOTAL_DATA_LIST:
        print("\n##### TOTAL_DATA {} #####".format(TOTAL_DATA))

        for EXPERIMENT_TYPE in EXPERIMENT_TYPE_LIST:
            for FOREIGN_LANG_DATA_MULT in FOREIGN_LANG_DATA_MULT_LIST:
                set_seed(seed=RANDOM_SEED)
                set_random_seed_data(seed=RANDOM_SEED)
                if EXPERIMENT_TYPE != 'C':
                    print('\n~~Result_{}_{}~~'.format(EXPERIMENT_TYPE,
                                                       TOTAL_DATA))
                else:
                    print('\n~~Result_{}_{}_{}~~'.format(EXPERIMENT_TYPE,
                                                   TOTAL_DATA,
                                                   FOREIGN_LANG_DATA_MULT))

                train_loader, valid_loader, test_loader = load_experiment_features(DATA_PATH_INDO,
                                                                                   DATA_PATH_FOREIGN,
                                                                                   tipe=EXPERIMENT_TYPE, 
                                                                                   total_data=TOTAL_DATA, 
                                                                                   foreign_mult=FOREIGN_LANG_DATA_MULT,
                                                                                   valid_size=VALIDATION_DATA)

                train(train_loader, valid_loader, learning_rate=0.0001)
                print("## Test phase...")
                y_true, y_pred_proba = test(test_loader)
                max_f1, max_recall, max_precision, max_threshold = evaluate(y_true, 
                                                                            y_pred_proba,
                                                                            threshold=0.5)

                result = pd.DataFrame({
                    'y_true': y_true,
                    'y_pred_proba': y_pred_proba
                })
                
                OUTPUT_FOREIGN_LANG_DATA_MULT = FOREIGN_LANG_DATA_MULT
                if EXPERIMENT_TYPE == 'A':
                    OUTPUT_FOREIGN_LANG_DATA_MULT = 0
                elif EXPERIMENT_TYPE == 'B':
                    OUTPUT_FOREIGN_LANG_DATA_MULT = -1
                
                OUTPUT_TOTAL_FOREIGN_DATA = int(TOTAL_DATA*FOREIGN_LANG_DATA_MULT)
                if EXPERIMENT_TYPE == 'A':
                    OUTPUT_TOTAL_FOREIGN_DATA = 0
                elif EXPERIMENT_TYPE == 'B':
                    OUTPUT_TOTAL_FOREIGN_DATA = TOTAL_DATA

                result_list.append({
                    'type':EXPERIMENT_TYPE,
                    'total_data':TOTAL_DATA, 
                    'seed':RANDOM_SEED,
                    'foreign_mult':OUTPUT_FOREIGN_LANG_DATA_MULT,
                    'total_foreign_data':OUTPUT_TOTAL_FOREIGN_DATA,
                    'f1':max_f1,
                    'recall':max_recall,
                    'precision':max_precision
                })

                os.remove("model.pt")

                if EXPERIMENT_TYPE != 'C':
                    break
                    
    final_result = pd.DataFrame(result_list) 
    final_result = final_result.sort_values(by=['total_data', 'foreign_mult'])
    final_result.to_csv('final_{}_{}_{}_result_combined_seed_{}.csv'.format(INDO_DATA_NAME,
                                                                           FOREIGN_DATA_NAME,
                                                                           MODEL_NAME,
                                                                           RANDOM_SEED),
                        index=False)
    
    result_list_final.extend(result_list)


############################
### START RANDOM_SEED 4 ###
############################

##### TOTAL_DATA 500 #####

~~Result_A_500~~
EarlyStopping! Epoch 20
Last: 7 	T-Loss: 0.665101 	T-Acc: 0.628889 	V-Loss: 0.689803 	V-Acc: 0.560000
## Test phase...
Final test Loss: 0.702937
[Evaluate] Threshold argument set. Using 0.5 as threshold
[Evaluate] ##MAX## 
THRESHOLD: 0.500 	F1: 0.33548387 	Recall: 1.00000000 	Prec: 0.50485437

~~Result_B_500~~
EarlyStopping! Epoch 35
Last: 22 	T-Loss: 0.683150 	T-Acc: 0.582222 	V-Loss: 0.683591 	V-Acc: 0.580000
## Test phase...
Final test Loss: 0.693706
[Evaluate] Threshold argument set. Using 0.5 as threshold
[Evaluate] ##MAX## 
THRESHOLD: 0.500 	F1: 0.37651332 	Recall: 0.90865385 	Prec: 0.49736842

~~Result_C_500_0.25~~
EarlyStopping! Epoch 87
Last: 74 	T-Loss: 0.626601 	T-Acc: 0.616341 	V-Loss: 0.668507 	V-Acc: 0.560000
## Test phase...
Final test Loss: 0.722034
[Evaluate] Threshold argument set. Using 0.5 as threshold
[Evaluate] ##MAX## 
THRESHOLD: 0.

/opt/conda/lib/python3.6/site-packages/torch/nn/modules/loss.py:498: UserWarning: Using a target size (torch.Size([1])) that is different to the input size (torch.Size([])) is deprecated. Please ensure they have the same size.
  return F.binary_cross_entropy(input, target, weight=self.weight, reduction=self.reduction)


EarlyStopping! Epoch 147
Last: 134 	T-Loss: 0.401753 	T-Acc: 0.826196 	V-Loss: 0.341272 	V-Acc: 0.863388
## Test phase...
Final test Loss: 0.575570
[Evaluate] Threshold argument set. Using 0.5 as threshold
[Evaluate] ##MAX## 
THRESHOLD: 0.500 	F1: 0.66255479 	Recall: 0.43750000 	Prec: 0.86666667

~~Result_C_10981_0.5~~
EarlyStopping! Epoch 94
Last: 81 	T-Loss: 0.429573 	T-Acc: 0.811846 	V-Loss: 0.351850 	V-Acc: 0.865209
## Test phase...
Final test Loss: 0.570289
[Evaluate] Threshold argument set. Using 0.5 as threshold
[Evaluate] ##MAX## 
THRESHOLD: 0.500 	F1: 0.67593814 	Recall: 0.46634615 	Prec: 0.85840708

~~Result_C_10981_0.75~~
EarlyStopping! Epoch 75
Last: 62 	T-Loss: 0.449706 	T-Acc: 0.808662 	V-Loss: 0.360477 	V-Acc: 0.861566
## Test phase...
Final test Loss: 0.577172
[Evaluate] Threshold argument set. Using 0.5 as threshold
[Evaluate] ##MAX## 
THRESHOLD: 0.500 	F1: 0.67443799 	Recall: 0.45673077 	Prec: 0.87155963

~~Result_C_10981_1~~
EarlyStopping! Epoch 115
Last: 102 	T-Loss

/opt/conda/lib/python3.6/site-packages/torch/nn/modules/loss.py:498: UserWarning: Using a target size (torch.Size([1])) that is different to the input size (torch.Size([])) is deprecated. Please ensure they have the same size.
  return F.binary_cross_entropy(input, target, weight=self.weight, reduction=self.reduction)


EarlyStopping! Epoch 52
Last: 39 	T-Loss: 0.430006 	T-Acc: 0.826248 	V-Loss: 0.372463 	V-Acc: 0.838798
## Test phase...
Final test Loss: 0.510928
[Evaluate] Threshold argument set. Using 0.5 as threshold
[Evaluate] ##MAX## 
THRESHOLD: 0.500 	F1: 0.77691699 	Recall: 0.67307692 	Prec: 0.85889571

~~Result_C_10981_6~~
EarlyStopping! Epoch 71
Last: 58 	T-Loss: 0.404728 	T-Acc: 0.832379 	V-Loss: 0.366106 	V-Acc: 0.841530
## Test phase...
Final test Loss: 0.484312
[Evaluate] Threshold argument set. Using 0.5 as threshold
[Evaluate] ##MAX## 
THRESHOLD: 0.500 	F1: 0.79517045 	Recall: 0.72115385 	Prec: 0.85227273

~~Result_C_10981_7~~
EarlyStopping! Epoch 73
Last: 60 	T-Loss: 0.389591 	T-Acc: 0.836826 	V-Loss: 0.361461 	V-Acc: 0.844262
## Test phase...
Final test Loss: 0.469377
[Evaluate] Threshold argument set. Using 0.5 as threshold
[Evaluate] ##MAX## 
THRESHOLD: 0.500 	F1: 0.80938145 	Recall: 0.72115385 	Prec: 0.88235294

~~Result_C_10981_8~~
EarlyStopping! Epoch 93
Last: 80 	T-Loss: 0.38839

...

EarlyStopping! Epoch 102
Last: 89 	T-Loss: 0.407213 	T-Acc: 0.831088 	V-Loss: 0.362995 	V-Acc: 0.851548
## Test phase...
Final test Loss: 0.489826
[Evaluate] Threshold argument set. Using 0.5 as threshold
[Evaluate] ##MAX## 
THRESHOLD: 0.500 	F1: 0.79376430 	Recall: 0.68269231 	Prec: 0.88750000

~~Result_C_10981_6~~
EarlyStopping! Epoch 54
Last: 41 	T-Loss: 0.415764 	T-Acc: 0.828289 	V-Loss: 0.370755 	V-Acc: 0.842441
## Test phase...
Final test Loss: 0.494766
[Evaluate] Threshold argument set. Using 0.5 as threshold
[Evaluate] ##MAX## 
THRESHOLD: 0.500 	F1: 0.79055140 	Recall: 0.72596154 	Prec: 0.83888889

~~Result_C_10981_7~~
EarlyStopping! Epoch 81
Last: 68 	T-Loss: 0.406504 	T-Acc: 0.831881 	V-Loss: 0.373174 	V-Acc: 0.838798
## Test phase...
Final test Loss: 0.488302
[Evaluate] Threshold argument set. Using 0.5 as threshold
[Evaluate] ##MAX## 
THRESHOLD: 0.500 	F1: 0.79251465 	Recall: 0.71153846 	Prec: 0.85549133

~~Result_C_10981_8~~
EarlyStopping! Epoch 57
Last: 44 	T-Loss: 0.3967

/opt/conda/lib/python3.6/site-packages/torch/nn/modules/loss.py:498: UserWarning: Using a target size (torch.Size([1])) that is different to the input size (torch.Size([])) is deprecated. Please ensure they have the same size.
  return F.binary_cross_entropy(input, target, weight=self.weight, reduction=self.reduction)


EarlyStopping! Epoch 144
Last: 131 	T-Loss: 0.395753 	T-Acc: 0.829029 	V-Loss: 0.331052 	V-Acc: 0.873406
## Test phase...
Final test Loss: 0.569988
[Evaluate] Threshold argument set. Using 0.5 as threshold
[Evaluate] ##MAX## 
THRESHOLD: 0.500 	F1: 0.66770662 	Recall: 0.44230769 	Prec: 0.87619048

~~Result_C_10981_0.5~~
EarlyStopping! Epoch 105
Last: 92 	T-Loss: 0.423810 	T-Acc: 0.816299 	V-Loss: 0.348570 	V-Acc: 0.862477
## Test phase...
Final test Loss: 0.562108
[Evaluate] Threshold argument set. Using 0.5 as threshold
[Evaluate] ##MAX## 
THRESHOLD: 0.500 	F1: 0.68464198 	Recall: 0.48076923 	Prec: 0.86206897

~~Result_C_10981_0.75~~
EarlyStopping! Epoch 134
Last: 121 	T-Loss: 0.414233 	T-Acc: 0.825836 	V-Loss: 0.333311 	V-Acc: 0.867942
## Test phase...
Final test Loss: 0.534266
[Evaluate] Threshold argument set. Using 0.5 as threshold
[Evaluate] ##MAX## 
THRESHOLD: 0.500 	F1: 0.71694271 	Recall: 0.52403846 	Prec: 0.89344262

~~Result_C_10981_1~~
EarlyStopping! Epoch 106
Last: 93 	T-Lo

/opt/conda/lib/python3.6/site-packages/torch/nn/modules/loss.py:498: UserWarning: Using a target size (torch.Size([1])) that is different to the input size (torch.Size([])) is deprecated. Please ensure they have the same size.
  return F.binary_cross_entropy(input, target, weight=self.weight, reduction=self.reduction)


EarlyStopping! Epoch 73
Last: 60 	T-Loss: 0.405563 	T-Acc: 0.832723 	V-Loss: 0.363086 	V-Acc: 0.843352
## Test phase...
Final test Loss: 0.486326
[Evaluate] Threshold argument set. Using 0.5 as threshold
[Evaluate] ##MAX## 
THRESHOLD: 0.500 	F1: 0.80143161 	Recall: 0.69711538 	Prec: 0.88957055

~~Result_C_10981_6~~
EarlyStopping! Epoch 58
Last: 45 	T-Loss: 0.420141 	T-Acc: 0.826829 	V-Loss: 0.381545 	V-Acc: 0.832423
## Test phase...
Final test Loss: 0.502607
[Evaluate] Threshold argument set. Using 0.5 as threshold
[Evaluate] ##MAX## 
THRESHOLD: 0.500 	F1: 0.77967914 	Recall: 0.68269231 	Prec: 0.85542169

~~Result_C_10981_7~~
EarlyStopping! Epoch 45
Last: 32 	T-Loss: 0.423634 	T-Acc: 0.825468 	V-Loss: 0.381264 	V-Acc: 0.833333
## Test phase...
Final test Loss: 0.503968
[Evaluate] Threshold argument set. Using 0.5 as threshold
[Evaluate] ##MAX## 
THRESHOLD: 0.500 	F1: 0.77171017 	Recall: 0.74038462 	Prec: 0.79381443

~~Result_C_10981_8~~
EarlyStopping! Epoch 60
Last: 47 	T-Loss: 0.39993

## Save

In [4]:
final_result = pd.DataFrame(result_list_final) 
final_result = final_result.sort_values(by=['total_data', 'foreign_mult'])
final_result.to_csv('final_{}_{}_{}_result_combined.csv'.format(INDO_DATA_NAME,
                                                               FOREIGN_DATA_NAME,
                                                               MODEL_NAME),
                        index=False)


In [5]:
print(final_result.shape)
final_result.head()

(288, 8)


,type,total_data,seed,foreign_mult,total_foreign_data,f1,recall,precision
1,B,500,4,-1.0,500,0.376513,0.908654,0.497368
97,B,500,5,-1.0,500,0.340904,1.000000,0.506083
193,B,500,6,-1.0,500,0.332253,0.985577,0.501222
0,A,500,4,0.0,0,0.335484,1.000000,0.504854
96,A,500,5,0.0,0,0.335484,1.000000,0.504854
